In [68]:
import re
import json

def parse_name(name):
    # Remove leading/trailing whitespace
    name = name.strip()

    # Define regular expression patterns for different name formats
    patterns = [
        r'^(?P<first>\w+)\s+(?P<middle>\w\.)\s+(?P<last>\w+)$',  # Firstname M. Lastname
        r'^(?P<first>\w+)\s+(?P<last>\w+)$',                     # Firstname Lastname
        r'^(?P<last>\w+),\s+(?P<first>\w+)$',                    # Lastname, Firstname
        r'^(?P<last>\w+),\s+(?P<first>\w+)\s+(?P<middle>\w\.)$'  # Lastname, Firstname M.
    ]

    # Iterate over the patterns and try to match the name
    for pattern in patterns:
        match = re.match(pattern, name)
        if match:
            # Extract the matched groups
            first = match.group('first')
            last = match.group('last')
            middle = match.group('middle') if 'middle' in match.groupdict() else ''

            # Capitalize the first letter and lowercase the rest
            first = first.capitalize()
            last = last.capitalize()
            middle = middle.capitalize() if middle else None

            return first, last, middle

    # If no match is found, return None for all fields
    return None, None, None

class Person:
    def __init__(self, name, people):
        # Parse name
        self.firstname, self.lastname, self.middle = parse_name(name)
        self.fullname = ' '.join(filter(None, [self.firstname, self.middle, self.lastname]))
        if self.fullname not in people['people']:
            people['people'][self.fullname] = self
        self.people = people

    def add_undirected_attribute(self, name, attribute):
        if attribute not in self.people['attributes']:
            self.people['attributes'][attribute] = 'undirected'
        target = Person(name, self.people)
        target_name = target.fullname
        if target_name in self.people['people']:
            target = self.people['people'][target_name]
        if not hasattr(self, attribute):
            setattr(self, attribute, [])
        if target_name not in getattr(self, attribute):
            getattr(self, attribute).append(target_name)
            target.add_undirected_attribute(self.fullname, attribute)

    def add_directed_attribute(self, name, attribute):
        if attribute not in self.people['attributes']:
            self.people['attributes'][attribute] = 'directed'
        target = Person(name, self.people)
        target_name = target.fullname
        if target_name in self.people:
            target = self.people[target_name]
        if not hasattr(self, attribute):
            setattr(self, attribute, [])
        if target_name not in getattr(self, attribute):
            getattr(self, attribute).append(target_name)

def load_people_from_file(filename, people = None):
    if not people:
        people = {'attributes': {}, 'people': {}}
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            people['attributes'] = data.get('attributes', {})
            for fullname, attrs in data.get('people', {}).items():
                person = Person(fullname, people)
                for attr, values in attrs.items():
                    setattr(person, attr, values)
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
    
    return people

def save_people_to_file(filename, people):
    data = {
        'attributes': people['attributes'],
        'people': {
            fullname: {
                attr: getattr(person, attr)
                for attr in dir(person)
                if not attr.startswith('__') and not callable(getattr(person, attr)) and attr != 'people'
            }
            for fullname, person in people['people'].items()
        }
    }
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

In [69]:
# Example usage
people = {'attributes': {}, 'people': {}}
john = Person('John Doe', people)
jane = Person('Jane Smith', people)
john.add_undirected_attribute('Jane Smith', 'friends')
john.add_directed_attribute('Hank Adams', 'children')
print(people)
save_people_to_file('test_people.json', people)

{'attributes': {'friends': 'undirected', 'children': 'directed'}, 'people': {'John Doe': <__main__.Person object at 0x1082e2240>, 'Jane Smith': <__main__.Person object at 0x1082e1730>, 'Hank Adams': <__main__.Person object at 0x1082e1be0>}}


In [71]:
persons = load_people_from_file('test_people.json')